In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from pandas.io import sql
import sqlite3

# Since 2010, which players have had the most game-winning hits?

Albert Pujols, Miguel Cabrera, and Joey Voto have had the most game-winning hits with 151, 130, and 128, respectively.

In [5]:
game_logs = pd.read_csv("../capstone/gl_csv/concat_game_logs.csv")

/var/folders/0b/5brsgfcj1659x_1js0_30dgm0000gn/T/ipykernel_17731/1136596188.py:1: DtypeWarning: Columns (13,14,15,85,87) have mixed types. Specify dtype option on import or set low_memory=False.
  game_logs = pd.read_csv("../capstone/gl_csv/concat_game_logs.csv")


In [6]:
game_logs_2010 = game_logs[game_logs["1 Date"] > 20100000]

In [7]:
game_logs_2010["101 Game Winning RBI player name"].value_counts().head(5)

(none)            1311
Albert Pujols      151
Miguel Cabrera     130
Joey Votto         128
Justin Upton       118
Name: 101 Game Winning RBI player name, dtype: int64

# Since 2017, which players have had the most game winning hits?

Nolan Arenado, Kyle Seagar, and Jose Ramirez have had the most game winning hits since 2017, with 60, 58, and 57, respectively. I do note that multiple players on the top 20 of this list belong to the same team. For example as of 2021, 3 belong to the Cardinals, 4 belong to the Dodgers, 4 belong to the Astros, 3 belong to the Phillies, 3 belong to the Braves, 2 belong to the Yankees.

In [8]:
game_logs_2017 = game_logs[game_logs["1 Date"] > 20170000]

In [9]:
game_logs_2017["101 Game Winning RBI player name"].value_counts().head(30)

(none)              501
Nolan Arenado        60
Kyle Seager          58
Jose Ramirez         57
Paul Goldschmidt     56
Bryce Harper         54
Aaron Judge          51
Carlos Correa        51
Alex Bregman         51
Kyle Schwarber       49
Cody Bellinger       49
Anthony Rendon       49
Freddie Freeman      48
Yulieski Gurriel     48
Matt Olson           47
Anthony Rizzo        47
Nelson Cruz          46
Jose Abreu           46
Jesus Aguilar        45
Yadier Molina        45
David Peralta        45
Manny Machado        44
Nick Castellanos     44
Marcell Ozuna        44
Christian Yelich     44
Jose Altuve          44
Ozzie Albies         43
J.D. Martinez        43
Joey Votto           43
George Springer      43
Name: 101 Game Winning RBI player name, dtype: int64

In [14]:
batting = pd.read_csv("../capstone/baseballdatabank-2022.2 2/core/batting.csv")

In [15]:
batting.head()

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,abercda01,1871,1,TRO,NaN,1,4,0,0,0,...,0.0,0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,0.0
1,addybo01,1871,1,RC1,NaN,25,118,30,32,6,...,13.0,8.0,1.0,4,0.0,NaN,NaN,NaN,NaN,0.0
2,allisar01,1871,1,CL1,NaN,29,137,28,40,4,...,19.0,3.0,1.0,2,5.0,NaN,NaN,NaN,NaN,1.0
3,allisdo01,1871,1,WS3,NaN,27,133,28,44,10,...,27.0,1.0,1.0,0,2.0,NaN,NaN,NaN,NaN,0.0
4,ansonca01,1871,1,RC1,NaN,25,120,29,39,11,...,16.0,6.0,2.0,2,1.0,NaN,NaN,NaN,NaN,0.0


In [16]:
batting_2017 = batting[batting["yearID"]>=2017]

In [16]:
batting_2017.sort_values(by = "H", ascending = False).head(20)

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
102956,blackch02,2017,1,COL,NL,159,644,137,213,35,...,104.0,14.0,10.0,65,135.0,9.0,10.0,3.0,3.0,4.0
106787,merriwh01,2019,1,KCA,AL,162,681,105,206,41,...,74.0,20.0,10.0,45,126.0,5.0,5.0,0.0,4.0,8.0
102864,altuvjo01,2017,1,HOU,AL,153,590,112,204,39,...,81.0,32.0,6.0,58,84.0,3.0,9.0,1.0,4.0,19.0
106217,deverra01,2019,1,BOS,AL,156,647,129,201,54,...,115.0,8.0,8.0,48,119.0,7.0,4.0,1.0,2.0,8.0
103492,inciaen01,2017,1,ATL,NL,157,662,93,201,27,...,57.0,22.0,9.0,49,94.0,3.0,0.0,3.0,4.0,8.0
103360,gordode01,2017,1,MIA,NL,158,653,114,201,20,...,33.0,60.0,16.0,25,93.0,0.0,10.0,2.0,4.0,7.0
106656,lemahdj01,2019,1,NYA,AL,145,602,109,197,33,...,102.0,5.0,2.0,46,90.0,0.0,2.0,1.0,4.0,14.0
103475,hosmeer01,2017,1,KCA,AL,162,603,98,192,31,...,94.0,6.0,1.0,66,104.0,3.0,0.0,0.0,2.0,20.0
105230,merriwh01,2018,1,KCA,AL,158,632,88,192,43,...,60.0,45.0,10.0,61,114.0,2.0,6.0,2.0,6.0,12.0
108937,bichebo01,2021,1,TOR,AL,159,640,121,191,30,...,102.0,25.0,1.0,40,137.0,0.0,6.0,0.0,4.0,10.0


# Which player has the most hits since 2017?

Whit Merrifield, Charlie Blackmon, and DJ LaMahieu have had the most hits since 2017, with 821, 783, and 764, respectively.

In [28]:
batting_2017.groupby("playerID")["H"].sum().sort_values(ascending = False).head(30)

playerID
merriwh01    821
blackch02    783
lemahdj01    764
freemfr01    755
arenano01    744
casteni01    743
altuvjo01    731
goldspa01    728
machama01    726
turnetr01    725
abreujo02    725
bogaexa01    711
bettsmo01    709
martijd02    702
gourryu01    700
lindofr01    696
segurje01    696
hosmeer01    689
anderti01    684
ramirjo01    676
hernace02    672
storytr01    667
yelicch01    658
semiema01    653
brantmi02    653
rizzoan01    634
baezja01     631
cruzne02     626
rosared01    625
escobed01    623
Name: H, dtype: int64

# Which player has the most walks since 2017?

Bryce Harper, Carlos Santana, and Joey Votto have had the most walks since 2017, with 446, 439, and 432, respectively.

In [27]:
batting_2017.groupby("playerID")["BB"].sum().sort_values(ascending = False).head(20)

playerID
harpebr03    446
santaca01    439
vottojo01    432
troutmi01    388
sotoju01     373
goldspa01    366
freemfr01    358
judgeaa01    352
bettsmo01    347
yelicch01    344
gallojo01    341
bregmal01    338
grandya01    338
carpema01    332
hoskirh01    316
mccutan01    314
ramirjo01    313
phamth01     312
rizzoan01    312
confomi01    308
Name: BB, dtype: int64

# Which players have been hit by the most pitches since 2017?

Anthony Rizzo, Mark Canha, and Derek Dietrich have been hit by the most pitches since 2017, with 104, 71, and 69 respectively.

In [21]:
batting_2017.groupby("playerID")["HBP"].sum().sort_values(ascending = False).head(10)

playerID
rizzoan01    104.0
canhama01     71.0
dietrde01     69.0
abreujo02     64.0
turneju01     63.0
bryankr01     60.0
wongko01      56.0
martest01     54.0
roblevi01     54.0
contrwi01     53.0
Name: HBP, dtype: float64

In [22]:
batting_2017["yearID"].value_counts()

2021    1706
2019    1569
2018    1535
2017    1494
2020    1360
Name: yearID, dtype: int64

# Which team has the most hits since 2017?

The Red Sox, Astros, and Colorado have had the most number of hits sicne 2017 with 6510, 6483, and 6296, respectively.

In [32]:
batting_2017.groupby("teamID")["H"].sum().sort_values(ascending = False).head(10)

teamID
BOS    6510
HOU    6483
COL    6296
WAS    6246
ATL    6195
MIN    6149
CHA    6094
NYA    6069
LAN    6008
TBA    5988
Name: H, dtype: int64

In [ ]:
# Identify the everyday players that are above average in batting categories over the past 5 seasons among other everyday players
# Identify the teams that are above average in batting categories over the past 5 seasons

# What is the average number of runs, hits, doubles, triples, homeruns, rbi, stolen bases, caught stealing, strike out, hit by pitch, sacrafice hits, sacrafice flies, and hit into double play for a player who has played in at least 100 games one or more times since 2017? 

An everyday player is defined as a player who played in at least 100 games each season.

In [38]:
batting_2017.head()

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
102831,abadfe01,2017,1,BOS,AL,48,0,0,0,0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
102832,abreujo02,2017,1,CHA,AL,156,621,95,189,43,...,102.0,3.0,0.0,35,119.0,6.0,15.0,0.0,4.0,21.0
102833,adamecr01,2017,1,COL,NL,12,13,1,0,0,...,0.0,0.0,0.0,1,6.0,0.0,0.0,0.0,0.0,0.0
102834,adamsau02,2017,1,WAS,NL,6,0,0,0,0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
102835,adamsla01,2017,1,ATL,NL,85,109,19,30,4,...,20.0,10.0,0.0,10,37.0,0.0,1.0,1.0,1.0,3.0


In [40]:
batting_2017.columns

Index(['playerID', 'yearID', 'stint', 'teamID', 'lgID', 'G', 'AB', 'R', 'H',
       '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'IBB', 'HBP', 'SH',
       'SF', 'GIDP'],
      dtype='object')

In [32]:
batting_2017_everyday_players = batting_2017[batting_2017["G"]>100]

In [33]:
batting_2017_offense_ctgs = batting_2017_everyday_players[['R', 'H',
       '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'HBP', 'SH',
       'SF', 'GIDP']]

In [34]:
batting_2017_offense_ctgs.describe()

,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,HBP,SH,SF,GIDP
count,923.000000,923.000000,923.000000,923.000000,923.000000,923.000000,923.000000,923.000000,923.000000,923.000000,923.000000,923.000000,923.000000,923.000000
mean,66.229686,120.276273,24.313109,2.278440,18.660888,63.521127,7.163597,2.560130,45.956663,106.997833,5.475623,0.884074,3.484290,9.876490
std,22.683843,35.172345,8.776108,2.171458,10.307362,23.763849,8.191068,2.552998,21.362745,34.331641,4.385962,1.710817,2.322791,4.961448
min,6.000000,10.000000,2.000000,0.000000,0.000000,5.000000,0.000000,0.000000,4.000000,17.000000,0.000000,0.000000,0.000000,0.000000
25%,50.000000,94.000000,18.000000,1.000000,11.000000,46.500000,2.000000,1.000000,30.000000,83.000000,2.000000,0.000000,2.000000,6.000000
50%,64.000000,120.000000,24.000000,2.000000,17.000000,62.000000,4.000000,2.000000,42.000000,106.000000,4.000000,0.000000,3.000000,9.000000
75%,82.000000,147.000000,30.000000,3.000000,25.000000,79.000000,10.000000,4.000000,59.000000,129.000000,7.000000,1.000000,5.000000,13.000000
max,137.000000,213.000000,56.000000,14.000000,59.000000,132.000000,60.000000,16.000000,145.000000,217.000000,27.000000,13.000000,12.000000,28.000000


In [35]:
stats = ['R', 'H', '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'HBP', 'SH', 'SF', 'GIDP']

batting_2017_ev_players_avg_stats = batting_2017_everyday_players.groupby("playerID")[stats].mean()

In [36]:
batting_2017_ev_players_avg_stats.head()

,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,HBP,SH,SF,GIDP
playerID,,,,,,,,,,,,,,
abreujo02,83.5,162.25,36.75,2.5,29.5,105.0,2.0,0.5,42.25,130.75,15.25,0.0,7.5,21.75
acunaro01,102.5,151.00,24.00,3.0,33.5,82.5,26.5,7.0,60.50,155.50,7.50,0.0,2.0,6.00
adamewi01,69.0,135.00,25.00,1.0,20.0,52.0,4.0,2.0,46.00,153.00,3.00,3.0,1.0,9.00
adamsma01,42.0,70.00,14.00,0.0,20.0,56.0,0.0,0.0,20.00,115.00,2.00,0.0,1.0,7.00
adriaeh01,37.0,64.50,16.00,1.5,5.5,33.5,2.5,0.5,22.50,62.00,1.50,2.5,2.5,4.00


# For players who have played in at least 100 games one or more times since 2017, which ones are above average in the offensive categories above, and below average in caught stealing, strike outs, and GIDP?

Interesting to note that Ketel Marte of the Arizona Diamondbacks is above average in all of the offensive categories except "SH", "SF", and "HBP", and below average in caught stealing, strikeouts, and GIDP. This means he had favorable stats in 11 of the 14 categories. But his sample size of 2 seasons in which he played more than 100 games, in the past five years, is smaller that the other players.

The only other player that is similar to Marte in grading out favorably in 11 of the 14 categories is Kyle Tucker of the Houston Astros. But Tucker's sample size is small as well: only 1 season in which he's played in more than 100 games in the past 5 years.

In [37]:
a = batting_2017_ev_players_avg_stats["R"] > 66
b = batting_2017_ev_players_avg_stats["H"] > 120
c = batting_2017_ev_players_avg_stats["2B"] > 24
d = batting_2017_ev_players_avg_stats["3B"] > 2
e = batting_2017_ev_players_avg_stats["HR"] > 18
f = batting_2017_ev_players_avg_stats["RBI"] > 63
g = batting_2017_ev_players_avg_stats["SB"] > 7
h = batting_2017_ev_players_avg_stats["CS"] < 3
i = batting_2017_ev_players_avg_stats["BB"] > 46
j = batting_2017_ev_players_avg_stats["SO"] < 107
k = batting_2017_ev_players_avg_stats["HBP"] > 5
l = batting_2017_ev_players_avg_stats["SH"] > 1
m = batting_2017_ev_players_avg_stats["SF"] > 3
n = batting_2017_ev_players_avg_stats["GIDP"] < 10 

In [97]:
batting_2017_ev_players_avg_stats[a & b & c & d & e & f & g & h & i & j & n]

,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,HBP,SH,SF,GIDP
playerID,,,,,,,,,,,,,,
marteke01,82.5,161.0,31.0,10.5,23.0,75.5,8.0,1.5,53.5,82.5,3.5,0.5,2.0,9.5


In [98]:
batting_2017_ev_players_avg_stats[a & b & c & d & e & f & g & h & i & j & m]

,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,HBP,SH,SF,GIDP
playerID,,,,,,,,,,,,,,
tuckeky01,83.0,149.0,37.0,3.0,30.0,92.0,14.0,2.0,53.0,90.0,1.0,0.0,5.0,10.0


# As of the 2021 season, which team has the highest number of players who have played in at least 100 games one or more times since 2017 and are above average in offensive categories (runs, hits, HR, RBI, BB) over the past 5 seasons?

AS of the 2021 season, the Padres, Astros, and Dodgers had the highest number of players who fit this criteria, each with 5. The Braves and Angels were next, with 4 players each, and then 10 teams are tied for third, each with 3 players.

In [50]:
batting_2017_ev_players_avg_stats[a & b & e & f & g & i & n]

,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,HBP,SH,SF,GIDP
playerID,,,,,,,,,,,,,,
acunaro01,102.500000,151.000000,24.000000,3.000000,33.500000,82.500000,26.500000,7.000000,60.500000,155.500000,7.500000,0.00,2.000000,6.000000
arozara01,94.000000,145.000000,32.000000,3.000000,20.000000,69.000000,20.000000,10.000000,56.000000,170.000000,14.000000,0.00,5.000000,9.000000
bellico01,97.333333,147.666667,29.333333,4.666667,37.000000,96.000000,13.000000,3.000000,76.000000,135.000000,2.333333,0.00,3.333333,7.333333
bettsmo01,114.500000,161.250000,40.500000,3.750000,27.000000,80.000000,20.500000,4.250000,80.750000,89.250000,6.000000,0.00,6.000000,7.500000
blackch02,111.000000,179.000000,33.250000,8.000000,27.750000,84.500000,7.750000,4.750000,54.500000,116.000000,9.500000,1.00,3.250000,8.250000
lindofr01,100.500000,158.750000,35.500000,2.750000,30.750000,79.500000,18.000000,5.500000,58.500000,98.500000,5.000000,3.75,3.750000,9.000000
marteke01,82.500000,161.000000,31.000000,10.500000,23.000000,75.500000,8.000000,1.500000,53.500000,82.500000,3.500000,0.50,2.000000,9.500000
meadoau01,81.000000,137.500000,29.000000,5.000000,30.000000,97.500000,8.000000,5.000000,56.500000,126.500000,6.500000,0.00,4.000000,3.500000
moncayo01,76.666667,144.666667,33.000000,4.000000,18.666667,67.000000,8.333333,3.666667,63.666667,176.000000,5.000000,1.00,2.333333,3.666667


In [51]:
test = batting_2017_ev_players_avg_stats[a & b & e & f & i]

In [52]:
test2 = [x for x in test.index]

In [53]:
batting_2017_ev_player_100games = batting_2017[batting_2017["playerID"].isin(test2)]

In [54]:
test3 = batting_2017_ev_player_100games[batting_2017_ev_player_100games["yearID"] == 2021].copy()

In [55]:
test3.drop_duplicates("playerID", inplace = True)

In [56]:
test3["teamID"].value_counts(ascending = False)

SDN    5
HOU    5
LAN    5
ATL    4
LAA    4
NYA    3
CIN    3
PHI    3
TOR    3
MIN    3
WAS    3
BOS    3
TBA    3
COL    3
MIL    3
NYN    2
OAK    2
CHN    2
CHA    2
SLN    2
KCA    2
ARI    2
TEX    1
SEA    1
BAL    1
CLE    1
PIT    1
Name: teamID, dtype: int64

In [61]:
test3[test3["teamID"] == "LAA"]

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
110040,rendoan01,2021,1,LAA,AL,58,217,24,52,13,...,34.0,0.0,0.0,29,41.0,2.0,1.0,0.0,2.0,11.0
110339,troutmi01,2021,1,LAA,AL,36,117,23,39,8,...,18.0,2.0,0.0,27,41.0,5.0,2.0,0.0,0.0,0.0
110352,uptonju01,2021,1,LAA,AL,89,318,47,67,12,...,41.0,4.0,1.0,39,107.0,0.0,1.0,0.0,3.0,11.0
110401,walshja01,2021,1,LAA,AL,144,530,70,147,34,...,98.0,2.0,1.0,48,152.0,6.0,4.0,0.0,3.0,7.0


# What is the average number of runs, hits, doubles, triples, homeruns, rbi, stolen bases, caught stealing, strike out, hit by pitch, sacrafice hits, sacrafice flies, and hit into double play for a player who has played in at least 500 games since 2017? 

In [177]:
# Groupby players who have played in more than 500 games in the past 5 years
a = batting_2017.groupby("playerID")["G"].sum() >500

In [178]:
# Turning groupby results into a dataframe
b = a.to_frame()

In [179]:
# used to following article to help me reset index of the dataframe
# https://sparkbyexamples.com/pandas/pandas-set-index-to-column-in-dataframe/#:~:text=In%20order%20to%20set%20index,row%20of%20the%20pandas%20DataFrame.
b.reset_index(inplace = True)

In [182]:
# Filtering the dataframe to identify the players who have played in more than 500 games in the past 5 years
b_true = b[b["G"] == True]

In [185]:
# Creating a playerID list of the players who have played in at least 500 games in the past 5 years. 
ev_player_500games_list = [x for x in b_true["playerID"]]

In [186]:
# Used the following stackoverflow article to filter rows using values in a list
# https://stackoverflow.com/questions/12065885/filter-dataframe-rows-if-value-in-column-is-in-a-set-list-of-values
batting_2017_ev_player_500games = batting_2017[batting_2017["playerID"].isin(ev_player_500games_list)]

In [187]:
batting_2017_ev_player_500games["G"].mean()

108.18385650224215

In [188]:
batting_2017_ev_player_500games_offense_ctgs = batting_2017_ev_player_500games[['R', 'H',
       '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'HBP', 'SH',
       'SF', 'GIDP']]

In [189]:
batting_2017_ev_player_500games_offense_ctgs.describe()

,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,HBP,SH,SF,GIDP
count,669.000000,669.000000,669.000000,669.000000,669.000000,669.000000,669.000000,669.000000,669.000000,669.000000,669.000000,669.000000,669.000000,669.000000
mean,57.860987,102.974589,21.112108,1.937220,16.665172,55.346786,6.366218,2.198804,39.916293,86.757848,4.452915,0.512706,3.026906,8.375187
std,29.882606,50.912490,11.614607,2.092674,10.848784,29.552033,7.821672,2.447215,24.414010,43.837936,3.963110,1.135401,2.497159,5.473238
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000
25%,32.000000,54.000000,11.000000,0.000000,8.000000,30.000000,1.000000,0.000000,21.000000,50.000000,2.000000,0.000000,1.000000,4.000000
50%,58.000000,108.000000,21.000000,1.000000,15.000000,55.000000,4.000000,1.000000,36.000000,86.000000,4.000000,0.000000,2.000000,8.000000
75%,81.000000,146.000000,30.000000,3.000000,24.000000,77.000000,9.000000,3.000000,55.000000,119.000000,6.000000,1.000000,4.000000,12.000000
max,137.000000,213.000000,56.000000,14.000000,52.000000,130.000000,59.000000,14.000000,134.000000,217.000000,27.000000,8.000000,12.000000,28.000000


# Of the players who have played in more than 500 games over the past 5 years, which players are above average in the offensive categories above, and below average in caught stealing, strike outs, and GIDP?

Of the players who have played in at least 500 games over the past 5 seasons, the Los Angeles Dodgers' Mookie Betts and the Cleveland Guardians' Jose Ramirez are above average in all offensive categories except "SH" and "SF", and below average in caught stealing, strikeouts, and GIDP.

In [489]:
stats = ['R', 'H', '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'HBP', 'SH', 'SF', 'GIDP']

batting_2017_ev_players_avg_stats_500games = batting_2017_ev_player_500games.groupby("playerID")[stats].mean()

In [510]:
a = batting_2017_ev_players_avg_stats_500games["R"] > 58
b = batting_2017_ev_players_avg_stats_500games["H"] > 103
c = batting_2017_ev_players_avg_stats_500games["2B"] > 21
d = batting_2017_ev_players_avg_stats_500games["3B"] > 2
e = batting_2017_ev_players_avg_stats_500games["HR"] > 17
f = batting_2017_ev_players_avg_stats_500games["RBI"] > 55
g = batting_2017_ev_players_avg_stats_500games["SB"] > 6
h = batting_2017_ev_players_avg_stats_500games["CS"] < 2
i = batting_2017_ev_players_avg_stats_500games["BB"] > 40
j = batting_2017_ev_players_avg_stats_500games["SO"] < 87
k = batting_2017_ev_players_avg_stats_500games["HBP"] > 4
l = batting_2017_ev_players_avg_stats_500games["SH"] > 1
m = batting_2017_ev_players_avg_stats_500games["SF"] > 3
n = batting_2017_ev_players_avg_stats_500games["GIDP"] < 8

In [491]:
batting_2017_ev_players_avg_stats_500games[a & b & c & d & e & f & g & i & j]

,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,HBP,SH,SF,GIDP
playerID,,,,,,,,,,,,,,
bettsmo01,101.0,141.8,34.2,3.2,24.8,71.8,18.4,3.8,69.4,79.0,5.2,0.0,5.0,6.4
ramirjo01,88.2,135.2,35.0,3.8,28.8,84.0,22.4,4.4,62.6,70.6,4.6,0.0,4.6,7.6


# Which teams in the 2021 season had the highest number of players who are above average over the past 5 years in offensive categories (runs, hits, HR, RBI, BB)?

In 2021, the Los Angeles Dodgers had the most players who fit this criteria with three, and nine other teams were tied with two players that met this criteria.

- LAN    3
- HOU    2
- ATL    2
- CIN    2
- TOR    2
- SLN    2
- SDN    2
- NYN    2
- BOS    2
- COL    2

In [525]:
test = batting_2017_ev_players_avg_stats_500games[a & b & e & f & i]

In [527]:
test2 = [x for x in test.index]

In [529]:
batting_ev_player_500games_atleastone_ab_avg = batting_2017[batting_2017["playerID"].isin(test2)]

In [530]:
test3 = batting_ev_player_500games_atleastone_ab_avg[batting_ev_player_500games_atleastone_ab_avg["yearID"] == 2021].copy()

In [531]:
test3.drop_duplicates("playerID", inplace = True)

In [533]:
test3["teamID"].value_counts()

LAN    3
HOU    2
ATL    2
CIN    2
TOR    2
SLN    2
SDN    2
NYN    2
BOS    2
COL    2
MIN    1
PHI    1
NYA    1
BAL    1
CLE    1
LAA    1
CHN    1
KCA    1
SEA    1
WAS    1
MIL    1
Name: teamID, dtype: int64

# EDA into Starting Pitcher Stats

In [2]:
pitching = pd.read_csv("../capstone/baseballdatabank-2022.2 2/core/pitching.csv")

In [3]:
pitching.head()

,playerID,yearID,stint,teamID,lgID,W,L,G,GS,CG,...,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP
0,bechtge01,1871,1,PH1,NaN,1,2,3,3,2,...,NaN,7,NaN,0,146.0,0,42,NaN,NaN,NaN
1,brainas01,1871,1,WS3,NaN,12,15,30,30,30,...,NaN,7,NaN,0,1291.0,0,292,NaN,NaN,NaN
2,fergubo01,1871,1,NY2,NaN,0,0,1,0,0,...,NaN,2,NaN,0,14.0,0,9,NaN,NaN,NaN
3,fishech01,1871,1,RC1,NaN,4,16,24,24,22,...,NaN,20,NaN,0,1080.0,1,257,NaN,NaN,NaN
4,fleetfr01,1871,1,NY2,NaN,0,1,1,1,1,...,NaN,0,NaN,0,57.0,0,21,NaN,NaN,NaN


In [10]:
pitching.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49430 entries, 0 to 49429
Data columns (total 30 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   playerID  49430 non-null  object 
 1   yearID    49430 non-null  int64  
 2   stint     49430 non-null  int64  
 3   teamID    49430 non-null  object 
 4   lgID      49298 non-null  object 
 5   W         49430 non-null  int64  
 6   L         49430 non-null  int64  
 7   G         49430 non-null  int64  
 8   GS        49430 non-null  int64  
 9   CG        49430 non-null  int64  
 10  SHO       49430 non-null  int64  
 11  SV        49430 non-null  int64  
 12  IPouts    49430 non-null  int64  
 13  H         49430 non-null  int64  
 14  ER        49430 non-null  int64  
 15  HR        49430 non-null  int64  
 16  BB        49430 non-null  int64  
 17  SO        49430 non-null  int64  
 18  BAOpp     44989 non-null  float64
 19  ERA       49333 non-null  float64
 20  IBB       34852 non-null  fl

In [11]:
pitching_2017 = pitching[pitching["yearID"] >= 2017]

# Which starting pitchers have started the most games since 2017?

Patrick Corbin, Gerritt Cole, Zack Greinke, and Aaron Nola have started the most games since 2017, with 140, 140, 139, and 138, respectively.

In [67]:
pitching_2017.groupby("playerID")["GS"].sum().sort_values(ascending = False).head(200)

playerID
corbipa01    140
colege01     140
greinza01    139
nolaaa01     138
lynnla01     136
            ... 
gombeau01     38
peacobr01     38
hernael01     38
mizeca01      37
suterbr01     37
Name: GS, Length: 200, dtype: int64

# What is the average pitching stats of the starting pitchers who have had 20 or more starts in at least one season over the past 5 seasons?

See descriptive stats below. For example, the average number of wins and losses is 10 and 9 (rounded up), respectively. Average earned run avergae (ERA) is 4.11, and bases-on-balls (BB) (aks walks) is 50.

In [105]:
pitching_2017_starters_20games = pitching_2017[pitching_2017["GS"] >= 20]

In [106]:
pitching_2017_starters_20games.describe().T

,count,mean,std,min,25%,50%,75%,max
yearID,457.0,2018.698031,1.475181,2017.000,2017.000,2018.000,2019.000,2021.000
stint,457.0,1.002188,0.046778,1.000,1.000,1.000,1.000,2.000
W,457.0,9.816193,3.880496,0.000,7.000,10.000,12.000,21.000
L,457.0,8.652079,3.082093,2.000,6.000,9.000,11.000,17.000
G,457.0,28.702407,4.493057,20.000,25.000,29.000,32.000,59.000
GS,457.0,27.404814,4.244219,20.000,24.000,28.000,31.000,35.000
CG,457.0,0.361050,0.706217,0.000,0.000,0.000,1.000,5.000
SHO,457.0,0.192560,0.470756,0.000,0.000,0.000,0.000,3.000
SV,457.0,0.039387,0.261955,0.000,0.000,0.000,0.000,3.000
IPouts,457.0,465.792123,91.036855,167.000,390.000,468.000,536.000,669.000


In [107]:
pitching_2017_starters_20games.columns

Index(['playerID', 'yearID', 'stint', 'teamID', 'lgID', 'W', 'L', 'G', 'GS',
       'CG', 'SHO', 'SV', 'IPouts', 'H', 'ER', 'HR', 'BB', 'SO', 'BAOpp',
       'ERA', 'IBB', 'WP', 'HBP', 'BK', 'BFP', 'GF', 'R', 'SH', 'SF', 'GIDP'],
      dtype='object')

In [108]:
stats = ['W', 'L', 'GS', 'CG', 'SHO', 'H', 'ER', 'HR', 'BB', 'SO', 'BAOpp', 'ERA', 'WP', 'HBP', 'SH', 'SF', 'GIDP']

pitching_2017_starters_20games_avg_stats = pitching_2017_starters_20games.groupby("playerID")[stats].mean()

In [109]:
pitching_2017_starters_20games_avg_stats.head().T

playerID,adlemti01,alcansa01,alzolad01,anderbr04,anderch01
W,5.000,7.500,5.000,8.5000,9.666667
L,11.000,14.500,13.000,9.0000,5.333333
GS,20.000,32.500,21.000,27.5000,27.333333
CG,0.000,1.500,0.000,0.0000,0.000000
SHO,0.000,1.000,0.000,0.0000,0.000000
H,124.000,175.000,112.000,141.5000,123.333333
ER,75.000,79.000,64.000,60.5000,59.000000
HR,29.000,22.000,25.000,15.5000,22.333333
BB,51.000,65.500,34.000,38.5000,49.333333
SO,108.000,176.000,128.000,74.0000,128.333333


# For the starting pitchers with 20 or more starts in at least one season over the past five seasons, which ones are better than average in hits allowed, home runs, walks, strikeouts, opposition batting average, and ERA?

The fourteenth starting pitchers listed below fit this criteria.

In [110]:
pitching_2017_starters_20games_avg_stats.columns

Index(['W', 'L', 'GS', 'CG', 'SHO', 'H', 'ER', 'HR', 'BB', 'SO', 'BAOpp',
       'ERA', 'WP', 'HBP', 'SH', 'SF', 'GIDP'],
      dtype='object')

In [111]:
a= pitching_2017_starters_20games_avg_stats["W"] > 10
b=pitching_2017_starters_20games_avg_stats["L"] < 9
c=pitching_2017_starters_20games_avg_stats["GS"] > 27
d=pitching_2017_starters_20games_avg_stats["CG"] > 1
e=pitching_2017_starters_20games_avg_stats["SHO"] > 0 
f=pitching_2017_starters_20games_avg_stats["H"] < 146
g=pitching_2017_starters_20games_avg_stats["ER"] < 70
h=pitching_2017_starters_20games_avg_stats["HR"] < 21
i=pitching_2017_starters_20games_avg_stats["BB"] < 49
j=pitching_2017_starters_20games_avg_stats["SO"] > 148
k=pitching_2017_starters_20games_avg_stats["BAOpp"] < 0.247
l=pitching_2017_starters_20games_avg_stats["ERA"] < 4.11
m=pitching_2017_starters_20games_avg_stats["WP"] < 5
n=pitching_2017_starters_20games_avg_stats["HBP"] < 6

o=pitching_2017_starters_20games_avg_stats["SH"] < 3
p=pitching_2017_starters_20games_avg_stats["SF"] < 4
q=pitching_2017_starters_20games_avg_stats["GIDP"] > 12

In [112]:
pitching_2017_starters_20games_avg_stats[f & h & i & j & k & l]

,W,L,GS,CG,SHO,H,ER,HR,BB,SO,BAOpp,ERA,WP,HBP,SH,SF,GIDP
playerID,,,,,,,,,,,,,,,,,
bassich01,11.000000,4.500000,26.000000,0.500000,0.500000,126.000000,58.000000,18.000000,43.000000,150.000000,0.223500,3.480000,4.000000,12.000000,1.000000,4.500000,8.500000
berrijo01,11.750000,8.000000,27.250000,1.000000,0.250000,144.750000,68.500000,20.000000,48.000000,165.500000,0.231250,3.722500,5.000000,10.750000,2.250000,4.250000,10.500000
buehlwa01,12.666667,4.333333,28.666667,0.666667,0.000000,132.333333,54.333333,17.000000,42.000000,192.666667,0.205000,2.783333,4.333333,6.333333,3.666667,4.000000,10.000000
burneco01,11.000000,5.000000,28.000000,0.000000,0.000000,123.000000,45.000000,7.000000,34.000000,234.000000,0.201000,2.430000,5.000000,6.000000,1.000000,4.000000,7.000000
hillri01,11.500000,6.500000,24.500000,0.500000,0.000000,103.500000,52.000000,19.000000,45.000000,158.000000,0.211000,3.490000,2.000000,8.500000,4.000000,1.500000,6.000000
kershcl01,13.250000,5.500000,25.750000,0.250000,0.000000,130.750000,50.500000,20.750000,30.250000,172.500000,0.221250,2.905000,7.000000,1.750000,4.000000,1.500000,12.000000
ohtansh01,9.000000,2.000000,23.000000,0.000000,0.000000,98.000000,46.000000,15.000000,44.000000,156.000000,0.207000,3.180000,10.000000,10.000000,2.000000,4.000000,8.000000
paxtoja01,12.666667,5.666667,27.000000,0.666667,0.333333,128.333333,58.666667,18.333333,44.666667,183.333333,0.229667,3.520000,10.000000,2.000000,1.666667,3.666667,9.000000
priceda01,11.500000,6.000000,26.000000,0.500000,0.000000,130.000000,60.500000,20.000000,41.000000,152.500000,0.244000,3.930000,2.000000,6.500000,0.500000,2.500000,8.500000


In [113]:
test = pitching_2017_starters_20games_avg_stats[f & h & i & j & k & l]

In [115]:
test2 = [x for x in test.index]
pitching_2017_starters_20games_ab_avg = pitching_2017[pitching_2017["playerID"].isin(test2)]
test3 = pitching_2017_starters_20games_avg_stats[pitching_2017_starters_20games_avg_stats["yearID"] == 2021].copy()
test3.drop_duplicates("playerID", inplace = True)
test3["teamID"].value_counts(ascending = False)

KeyError: 'yearID'

In [ ]:
# The relief pitchers are all the pitchers with five or less "GS"

In [70]:
pitching_2017_relief = pitching_2017[pitching_2017["GS"] < 5]

In [534]:
# EDA in defense/fielding stats